OG Note: This demo is adapted from the LXMERT Demo present here: https://github.com/huggingface/transformers/tree/main/examples/research_projects/lxmert

JN Note: The bulk of the code was sourced from: https://github.com/huggingface/transformers/tree/main/examples/research_projects/visual_bert

In [1]:
# Basic packages
from IPython.display import Image, display
import PIL.Image
import io
import torch
import numpy as np
import json
import glob
import re
from os.path import exists
from tqdm import tqdm
import time
import random

random.seed(1)

# Local packages
from processing_image import Preprocess
from visualizing_image import SingleImageViz
from modeling_frcnn import GeneralizedRCNN
from utils import Config
import utils

# Model
from transformers import LxmertForQuestionAnswering, LxmertTokenizer

### INSTRUCTION FOR USERS : INDICATE APPROPRIATE PATH
output_title_base='/u/scr/nlp/data/nli-consistency/lxmert_results/lxmert-test-3pred-40token-1seed_predictions'

### INSTRUCTION FOR USERS : INDICATE APPROPRIATE PATH
# Data from https://visualgenome.org/api/v0/api_home.html
# Used Version 1.2 as that one had Part 1 and Part 2 (selected Part 1)
VG_path = '/u/scr/nlp/data/nli-consistency/vg_data/VG_100K/'
VG_path2 = '/u/scr/nlp/data/nli-consistency/vg_data/VG_100K_2/'

### INSTRUCTION FOR USERS : INDICATE APPROPRIATE PATH
# L-ConVQA from https://arijitray1993.github.io/ConVQA/
file_source='/u/scr/nlp/data/nli-consistency/cleaned_Logical_ConVQA_test.json'
file_paths=[]
with open(file_source, 'r') as stream:
    data = json.load(stream)
    
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'

### INSTRUCTION FOR USERS : INDICATE APPROPRIATE PATH
# VQA_URL = "https://raw.githubusercontent.com/airsplay/lxmert/master/data/vqa/trainval_label2ans.json"
VQA_URL = '/u/scr/nlp/data/nli-consistency/trainval_label2ans.json' # downloaded version from above

In [2]:
# load answer labels
vqa_answers = utils.get_data(VQA_URL)

def convert_multiple_answers(predictions, vqa_answers):
    return [vqa_answers[predictions[i]] for i in range(len(predictions))]

In [3]:
# load models and model components
frcnn_cfg = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")
frcnn_cfg.model.device = device

frcnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=frcnn_cfg)
image_preprocess = Preprocess(frcnn_cfg)

lxmert_tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
lxmert_vqa = LxmertForQuestionAnswering.from_pretrained("unc-nlp/lxmert-vqa-uncased").to(device)

loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at /sailhome/jnoh2/.cache/torch/transformers/57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [4]:
# Partially based on code from :
# https://huggingface.co/docs/transformers/model_doc/visual_bert

predictions = {}
num_predictions = 3

# for key in cleaned_data.keys():
#     image_num = key
#     image_path = VG_path + str(image_num) + '.jpg'
    
#     qas = cleaned_data[key]

counter = 0

no_image = []

start = time.process_time()

# It takes 4.1 seconds for 20 images; 5 images per second
# for key in data.keys():
data_keys = list(data.keys())
for i in tqdm(range(len(data_keys))):
    key = data_keys[i]
    image_num = key
    
    image_path1 = VG_path + str(image_num) + '.jpg'
    image_path2 = VG_path2 + str(image_num) + '.jpg'
    
    if exists(image_path1):
        image_path = image_path1
    elif exists(image_path2):
        image_path = image_path2
    else:
        no_image.append(image_num)
        continue
    
    # From the main demo
    images, sizes, scales_yx = image_preprocess(image_path)
    output_dict = frcnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=frcnn_cfg.max_detections,
        return_tensors="pt",
    )
    
    normalized_boxes = output_dict.get("normalized_boxes")
    features = output_dict.get("roi_features")
    # From the main demo
    
    predictions[image_num] = {}
    
    qas = data[key]
    
    # Sets of consistent questions
    for j in range(len(qas)):
        qs_only = [qas[j][k]['question'] for k in range(len(qas[j]))]
        inputs = lxmert_tokenizer(
            qs_only, 
            return_tensors='pt', 
            padding = "max_length", 
            max_length=40,
            truncation = True, 
            add_special_tokens = True, 
            return_token_type_ids=True, 
            return_attention_mask=True,
        )
        
        output_vqa = lxmert_vqa(
            input_ids=inputs.input_ids.to(device),
            attention_mask=inputs.attention_mask.to(device),
            visual_feats=features.expand(len(inputs.input_ids), -1, -1).to(device),
            visual_pos=normalized_boxes.expand(len(inputs.input_ids), -1, -1).to(device),
            token_type_ids=inputs.token_type_ids.to(device),
            output_attentions=False,
        )
        
        final_logits = output_vqa['question_answering_score']
        final_softmax = final_logits.softmax(dim = -1)
        final_sorted, final_indices = torch.sort(final_softmax, dim = -1, descending = True)
        
        predictions[image_num][j] = [qas[j][k]|{'prediction':convert_multiple_answers(final_indices[k][0:num_predictions], vqa_answers), 'prob':final_sorted[k][0:num_predictions].tolist()} for k in range(len(qas[j]))]
        
    counter += 1    
    
total_time = time.process_time() - start

  0%|                                                                                                       | 0/725 [00:00<?, ?it/s]/nlp/scr/jnoh2/miniconda3/envs/nli-consistency-vilt/lib/python3.9/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/nlp/scr/jnoh2/miniconda3/envs/nli-consistency-vilt/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755849709/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|█████████████████████████████████████████████████████████████████████████████████████████████| 725/725 [04:43<00:00, 

In [5]:
correct = 0
incorrect = 0

qa_correct = []
qa_incorrect = []

for key in predictions.keys():
    for set_num in predictions[key].keys():
        for qas in predictions[key][set_num]:
            if qas['answer'] != qas['prediction'][0]:
                incorrect += 1
                qa_incorrect.append((qas['answer'], qas['prediction'][0]))
            else:
                correct += 1
                qa_correct.append((qas['answer'], qas['prediction'][0]))

accuracy = float(correct) / (float(correct) + float(incorrect))

accuracy

0.6559028292104874

In [6]:
with open(output_title_base + '.json', 'w') as f:
    json.dump(predictions, f)
with open(output_title_base + '.txt', 'w') as f:
    json.dump({'accuracy: ':accuracy, 
               'qa_correct: ':len(qa_correct), 
               'qa_incorrect: ':len(qa_incorrect), 
               'no_image: ':len(no_image),
               'total_questions: ':len(qa_correct) + len(qa_incorrect) + len(no_image),
               'time_taken: ':total_time,
              }, f)

In [7]:
accuracy

0.6559028292104874

In [ ]:
predictions

In [8]:
total_time

468.385799099